#### Contours

In [16]:
import cv2
import numpy as np

image = cv2.imread("D:\Computer_Vision_Recap\images/shapes.jpg")

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edged  = cv2.Canny(gray, 30, 200)

# finding contours in image
contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print("Num of contours found: " + str(len(contours)))

# draws all contours, -1 as 3rd param draws all
cv2.drawContours(image, contours, -1, (255, 0, 0), 3)

cv2.imshow("Contours", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Num of contours found: 3


***
#### Sorting Contours

In [22]:
import cv2
import numpy as np

image = cv2.imread("D:\Computer_Vision_Recap\images/BunchOfShapes.jpg")
height, width, color = image.shape

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edged  = cv2.Canny(gray, 30, 200)

# black canvas for showing contours
black_canvas = np.zeros((height, width, 3),dtype ="uint8")

# finding contours in image
contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print("Num of contours found: " + str(len(contours)))

# draws all contours, -1 as 3rd param draws all on normal image
cv2.drawContours(image, contours, -1, (255, 0, 0), 3)
# draw contours on black_cavas
cv2.drawContours(black_canvas, contours, -1, (255, 0, 0), 3)

cv2.imshow("Contours", black_canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()

Num of contours found: 4


In [33]:
# ssorting contours by size or area
import cv2
import numpy as np

# func to display contour area and ret area of all contours as list
def get_contour_areas(contours):
    all_areas = []
    for contour in contours:
        area = cv2.contourArea(contour)      # gets area of the contour
        all_areas.append(area)
    return all_areas

# black canvas for showing contours
black_canvas = np.zeros((height, width, 3),dtype ="uint8")

# contour areas b4 sort
print("The contour areas b4 sort: " + str(get_contour_areas(contours)))

# sort contours large to small
sorted_contours = sorted(contours, key = cv2.contourArea, reverse = True)

# contour areas after sorting
print("The contour areas after sorting: " + str(get_contour_areas(sorted_contours)))


# iterate through contours and draw one by one
for i, cnt in enumerate(sorted_contours):
    cv2.drawContours(black_canvas, [cnt], -1, (0, 255, 0), 3)
    cv2.imshow("Contour by area for contour: {0}".format(i+1), black_canvas)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    
cv2.waitKey(0)
cv2.destroyAllWindows()

The contour areas b4 sort: [15581.0, 14012.0, 45142.0, 28269.0]
The contour areas after sorting: [45142.0, 28269.0, 15581.0, 14012.0]


In [42]:
# sorting contours from left to right
import cv2
import numpy as np


def x_cord_contour(contours):
    # func used to sorts contour by pos & ret X cord of send centroid
    if cv2.contourArea(contours) > 10:
        M = cv2.moments(contours)             # gets center point of the contours in bracket
        return (int(M["m10"]/M["m00"]))
    
def label_contour_center(image, contours):
    # places blue circle on centroid of contours
    M = cv2.moments(contours)
    cx = int(M["m10"]/M["m00"])
    cy = int(M["m01"]/M["m00"])
    
    # draw the circle on centroid
    cv2.circle(image, (cx, cy), 10, (255, 0, 0), -1)
    return image


# loads image and creates the black_canvas
image = cv2.imread("D:\Computer_Vision_Recap\images/BunchOfShapes.jpg")
original_image = image.copy()
height, width, color = image.shape
black_canvas = np.zeros((height, width, 3),dtype ="uint8")


# computers center of mass or centroid and draw on image
for i, c in enumerate(contours):
    orig = label_contour_center(image, c)

cv2.imshow("contour Centers ", image)
cv2.waitKey(0)


# sorts contours from left to right using x_cord_contour func
contours_left_to_right = sorted(contours, key = x_cord_contour, reverse=False)

# Labels contour from left to right
for i, contour in enumerate(contours_left_to_right):
    cv2.drawContours(black_canvas, [contour], -1, (255, 0, 0), 3)
    M = cv2.moments(contour)
    cx = int(M["m10"]/M["m00"])
    cy = int(M["m01"]/M["m00"])
    cv2.putText(black_canvas, str(i+1), (cx-5, cy-5), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
    cv2.imshow("Sorted from left to right", black_canvas)
    cv2.waitKey(0)
    
    # gets bounding box housing the contours
    (x1, y1, x2, y2) = cv2.boundingRect(contour)
    # crop each contour and save to disk
    cropped_contour = original_image[y1:y1 + y2, x1:x1 + x2]
    image_name = "output_shape_number " + str(i+1) + ".jpg"
    cv2.imwrite(image_name, cropped_contour)
    
cv2.destroyAllWindows()
    

****
#### Approximating Contours and Convex Hull
* approx contour shape better rather than just use bounding boxes.

In [52]:
import numpy as np
import cv2

# load image and keep a copy
image = cv2.imread("D:\Computer_Vision_Recap\images/House.jpg")
orig_image = image.copy()

# grayscales and bin image
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray_image, 125, 255, cv2.THRESH_BINARY_INV)

# finds contours 
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# iterate through each contour and get bounding box
for contour in contours:
    x1, y1, x2, y2 = cv2.boundingRect(contour)
    cv2.rectangle(orig_image, (x1, y1), (x1+x2, y1+y2), (255, 0, 0), 2)
    cv2.imshow("Bounding box", orig_image)
    
cv2.waitKey(0)

# iterate through each contour and get approx contour
for contour in contours:
    # calc accuracy as % of contour perimeter
    accuracy = 0.01* cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, accuracy, True)
    cv2.drawContours(image, [approx], 0, (225, 0, 0), 2)
    cv2.imshow("Approx Poly Dp", image)
    
cv2.waitKey(0)
cv2.destroyAllWindows()

In [71]:
# convex Hull(smallest polygon that can fit around object)
import numpy as np
import cv2

image = cv2.imread("D:\Computer_Vision_Recap\images/Star.jpg")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# thresh image and find contours
ret, thresh = cv2.threshold(gray_image, 170, 255, 0)
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# sorts contour by area and removes largest frame contour that's the image frame
n = len(contours) - 1
contours = sorted(contours, key = cv2.contourArea, reverse=False)[:n]

# iterates through contours and draw the convex hull
for contour in contours:
    hull = cv2.convexHull(contour)
    cv2.drawContours(image, [hull], 0, (255, 0, 0), 2)
    cv2.imshow("Convex Hull", image)
    
cv2.waitKey(0)
cv2.destroyAllWindows()

***
#### Matching Contours

In [107]:
import cv2
import numpy as np

# load shape to identify in list
template = cv2.imread("D:\Computer_Vision_Recap\images/Star_2.jpg", 0)
# loads the list of shapes
target = cv2.imread("D:\Computer_Vision_Recap\images/Shapes_to_match.jpg")
target_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)

# theresh both images to remove noise
ret, thresh1 = cv2.threshold(template, 127, 255, 0)
ret, thresh2 = cv2.threshold(target_gray, 127, 255, 0)

# finds and sort contours of template and remove largest contour, i.e image outline by xtracting  2nd largest
contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
sorted_contours = sorted(contours, key = cv2.contourArea, reverse=True)
template_contour = sorted_contours[1]       # gets 2nd largest contour


# gets contours of target list
target_contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

for contour in target_contours:
    # iterate through each contour in target and match with template contour
    match = cv2.matchShapes(template_contour, contour, 1, 0.0)
    print(match)
    if match < 0.30:      # the lesser the match value the more similar it is
        closest_contour = contour
    else:
        closest_contour = []

# draws contour over image that matches in list
cv2.drawContours(target, [closest_contour], -1, (0, 0, 255), 3)
cv2.imshow("matched_object", target)
cv2.waitKey(0)

cv2.destroyAllWindows()
            


0.29032591427461973
0.35307680545737763
0.30181643676514236
0.2735596255305207


#### Line Detection
* Hough Lines
* Probablisitic Hough Lines

n.b: Not performing well yet

In [5]:
# Line detection using Hough Lines
import cv2
import numpy as np

image = cv2.imread("D:/Computer_Vision_Recap/images/soduku.jpg")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# ret, thresh = cv2.threshold(gray_image, 20, 100, 0)
edges = cv2.Canny(gray_image, 100, 150, apertureSize = 3)

# get line using HoughLines
lines = cv2.HoughLines(edges, 1, np.pi / 180, 250)

# iterate through the lines detected in houghline format and change to cv.lines format
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    
cv2.imshow("Hough_Line Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
# Using Probablistic HoughLines
image = cv2.imread("D:/Computer_Vision_Recap/images/sudoku_2.jpg")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# ret, thresh = cv2.threshold(gray_image, 20, 100, 0)
edges = cv2.Canny(gray_image, 100, 170, apertureSize = 3)


# gets line using Probablistic HoughLine detection
lines = cv2.HoughLinesP(edges, 3, np.pi / 180, 350, 5, 7)
print(lines.shape)

# no need for preprocessing result here
for line in lines[0]:
    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 5)
    
cv2.imshow("Probablistic Hough_Line Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(182, 1, 4)


****
#### BlobDetection

In [12]:
import cv2
import numpy as np

image = cv2.imread("D:/Computer_Vision_Recap/images/sunflower.jpg")
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

detector = cv2.SimpleBlobDetector_create()       # init the detector
keypoints = detector.detect(gray_image)          # detects blobs

# draw detected blobs, n.b: cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
# ensures size of circle is actual sixe of blob

blank = np.zeros((1,1), np.uint8)
blobs = cv2.drawKeypoints(image, keypoints, blank, (0, 255, 0),
                         cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow("Blob Detection", blobs)
cv2.waitKey(0)
cv2.destroyAllWindows()
